## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-29-10-47-00 +0000,wsj,Thai Prime Minister Ousted Over Leaked Phone Call,https://www.wsj.com/world/thai-prime-minister-...
1,2025-08-29-10-45-48 +0000,bbc,P&O Ferries boss who sacked 800 staff quits,https://www.bbc.com/news/articles/cdrk8j81km2o...
2,2025-08-29-10-44-45 +0000,nypost,JD Vance says he’s ready to take over as presi...,https://nypost.com/2025/08/29/us-news/vance-re...
3,2025-08-29-10-40-41 +0000,bbc,UK blocks Israeli government delegation from a...,https://www.bbc.com/news/articles/cvgpxwy2lkwo...
4,2025-08-29-10-39-36 +0000,nypost,Moment feds descend on Minneapolis shooter Rob...,https://nypost.com/2025/08/29/us-news/moment-f...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2342/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
21,trump,50
178,new,19
604,shooting,16
797,fire,11
33,minneapolis,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
184,2025-08-28-21-40-00 +0000,wsj,"Fed governor Lisa Cook sued President Trump, s...",https://www.wsj.com/economy/central-banking/li...,139
287,2025-08-28-14-18-31 +0000,cbc,U.S. Fed governor Lisa Cook suing Trump admini...,https://www.cbc.ca/news/world/trump-lisa-cook-...,112
270,2025-08-28-15-24-39 +0000,bbc,US Fed Governor Lisa Cook sues Trump over his ...,https://www.bbc.com/news/articles/c1dxl6ry4y3o...,108
274,2025-08-28-15-14-52 +0000,bbc,Can Trump fire Fed Governor Lisa Cook and why ...,https://www.bbc.com/news/articles/cedvj2d5538o...,104
285,2025-08-28-14-29-51 +0000,nypost,Embattled Federal Reserve governor Lisa Cook s...,https://nypost.com/2025/08/28/us-news/embattle...,101


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
184,139,2025-08-28-21-40-00 +0000,wsj,"Fed governor Lisa Cook sued President Trump, s...",https://www.wsj.com/economy/central-banking/li...
141,56,2025-08-29-00-39-00 +0000,wsj,New Trial Ordered for Three Former Police Offi...,https://www.wsj.com/us-news/law/new-trial-orde...
205,54,2025-08-28-20-32-41 +0000,nypost,"UK, EU accuse Putin of ‘sabotaging peace’ afte...",https://nypost.com/2025/08/28/world-news/uk-eu...
167,51,2025-08-28-22-49-14 +0000,bbc,"'We love you, you will always be with us', say...",https://www.bbc.com/news/videos/cwy5zzvgj44o?a...
241,42,2025-08-28-16-56-16 +0000,latimes,ChatGPT pulled teen into a 'dark and hopeless ...,https://www.latimes.com/business/story/2025-08...
115,37,2025-08-29-02-16-31 +0000,nypost,RFK Jr’s deputy expected to replace ousted CDC...,https://nypost.com/2025/08/28/us-news/rfk-jrs-...
140,33,2025-08-29-00-41-41 +0000,nypost,One of 37 intel pros whose clearance was pulle...,https://nypost.com/2025/08/28/us-news/one-of-3...
170,32,2025-08-28-22-33-31 +0000,startribune,Families identify Fletcher Merkel and Harper M...,https://www.startribune.com/family-of-8-year-o...
295,30,2025-08-28-13-46-00 +0000,wsj,"President Trump fired Robert Primus, a board m...",https://www.wsj.com/politics/policy/trump-fire...
148,29,2025-08-29-00-28-26 +0000,nypost,Vance takes swipe at Mamdani — says Ugandan-bo...,https://nypost.com/2025/08/28/us-news/vance-sa...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
